In [1]:
import math
from os.path import join
from google.colab import drive

import numpy as np
import pandas as pd
import calendar
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.model_selection import train_test_split
import tensorflow as tf

from matplotlib import pyplot as plt

import pycountry_convert as pc

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load data
path = '/content/drive/MyDrive/Colab Notebooks/data/'
df = pd.read_csv(path + 'hotel_bookings.csv')
print(f'Original dataset shape: {df.shape}')

Original dataset shape: (119390, 32)


## Data Profile

In [4]:
df.info()
# Children, country, agent, and company have nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

## Data Cleansing and Processing

In [5]:
# Data processing - filling nulls and adjusting datatypes/values
df['agent'] = df['agent'].notna().astype(int)
df['arrival_date_month'] = pd.to_datetime(
    df['arrival_date_month'], format='%B').dt.month
df['children'] = df['children'].fillna(0)
df['company'] = df['company'].notna().astype(int)

# Map countries to continents
manual_map = {
    'CN': 'Asia',
    'TMP': 'Asia',       # Timor-Leste
    'UMI': 'Oceania',    # U.S. Minor Outlying Islands
    'ATA': 'Antarctica', # Antarctica
    'ATF': 'Antarctica', # French Southern Territories
    None: 'Unknown',
    float('nan'): 'Unknown'
}

def country_to_continent(country_code):
    try:
        if pd.isna(country_code) or country_code in manual_map:
            return manual_map.get(country_code, 'Unspecified')
        # conversion between iso
        country_alpha2 = pc.country_alpha3_to_country_alpha2(country_code)
        # iso convert to continent
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        return pc.convert_continent_code_to_continent_name(continent_code)
    except:
        return 'Unspecified'

df['continent'] = df['country'].apply(country_to_continent)

# Determine if customer got their desired room type
mask_equal = df['assigned_room_type'] == df['reserved_room_type']
mask_not_ik = ~df['assigned_room_type'].isin(['I', 'K'])
df['is_reserved_room_type'] = (mask_equal & mask_not_ik).astype(int)

# Drop outliers for ADR (greater than 5000 and negative values)
df = df[(df.adr >= 0) & (df.adr < 5000)]

# Drop unneeded columns
cleaned_df = df.drop(['country',
              'assigned_room_type',
              'arrival_date_day_of_month',
              'arrival_date_week_number',
              'arrival_date_year',
              'reservation_status',
              'reservation_status_date',
              'reserved_room_type'], axis=1)

In [6]:
cleaned_df.head()

,hotel,is_canceled,lead_time,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,continent,is_reserved_room_type
0,Resort Hotel,0,342,7,0,0,2,0.0,0,BB,...,No Deposit,0,0,0,Transient,0.0,0,0,Europe,1
1,Resort Hotel,0,737,7,0,0,2,0.0,0,BB,...,No Deposit,0,0,0,Transient,0.0,0,0,Europe,1
2,Resort Hotel,0,7,7,0,1,1,0.0,0,BB,...,No Deposit,0,0,0,Transient,75.0,0,0,Europe,0
3,Resort Hotel,0,13,7,0,1,1,0.0,0,BB,...,No Deposit,1,0,0,Transient,75.0,0,0,Europe,1
4,Resort Hotel,0,14,7,0,2,2,0.0,0,BB,...,No Deposit,1,0,0,Transient,98.0,0,1,Europe,1


## One-Hot Encoding

In [7]:
# One-hot encode categorical columns
encoded_df = pd.get_dummies(cleaned_df,
                    columns=['customer_type',
                             'deposit_type',
                             'distribution_channel',
                             'hotel',
                             'market_segment',
                             'meal',
                             'continent'],
                    dtype = int)

encoded_df.columns = encoded_df.columns.str.replace(r"[ _/\-]", '_', regex=True).str.lower()

print(f'Processed dataset shape: {encoded_df.shape}')
encoded_df.to_csv(path + 'hotel_bookings_processed.csv', index=False)

Processed dataset shape: (119388, 54)


## Splitting Data into Train/Validation/Test Sets

In [8]:
# Load processed file - index column included (55 columns)
processed_df = pd.read_csv(path + 'hotel_bookings_processed.csv')
print(f'Loaded processed dataset shape: {processed_df.shape}')

# Percentage of cancelled bookings
not_cxl = processed_df.is_canceled.value_counts()[0]
cxl = processed_df.is_canceled.value_counts()[1]
cxl_pct = cxl / (cxl + not_cxl)
print(f'Pct of cancelled bookings: {cxl_pct.item()}')

Loaded processed dataset shape: (119388, 54)
Pct of cancelled bookings: 0.37041411197105234


In [9]:
# Setting Y and X
Y = processed_df[['is_canceled']]
print(f"Shape of Y: {Y.shape}")

X = processed_df.drop(columns=['is_canceled'])
print(f"Shape of X: {X.shape}\n")

# Partition data into training (60%), validation (20%), and test (20%)
# Shuffle defaulted to true
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, shuffle=True, stratify=Y_train)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_val shape: {Y_val.shape}")
print(f"Y_test shape: {Y_test.shape}")

Shape of Y: (119388, 1)
Shape of X: (119388, 53)

X_train shape: (71632, 53)
X_val shape: (23878, 53)
X_test shape: (23878, 53)
Y_train shape: (71632, 1)
Y_val shape: (23878, 1)
Y_test shape: (23878, 1)


In [10]:
# Validate that stratified randomization was done correctly
for data, y in {'Y_train': Y_train, 'Y_val': Y_val, 'Y_test': Y_test}.items():
  Y_not_cxl = y.is_canceled.value_counts()[0]
  Y_cxl = y.is_canceled.value_counts()[1]
  Y_cxl_pct = Y_cxl / (Y_cxl + Y_not_cxl)
  print(f'After-split {data} cancelled bookings pct: {Y_cxl_pct.item()}')

After-split Y_train cancelled bookings pct: 0.37040708063435335
After-split Y_val cancelled bookings pct: 0.3704246586816316
After-split Y_test cancelled bookings pct: 0.3704246586816316


In [11]:
# Save train/val/test files in different CSV
X_train.to_csv(path + 'hotel_bookings_X_train.csv', index=False)
Y_train.to_csv(path + 'hotel_bookings_Y_train.csv', index=False)
X_val.to_csv(path + 'hotel_bookings_X_val.csv', index=False)
Y_val.to_csv(path + 'hotel_bookings_Y_val.csv', index=False)
X_test.to_csv(path + 'hotel_bookings_X_test.csv', index=False)
Y_test.to_csv(path + 'hotel_bookings_Y_test.csv', index=False)